In [1]:
import os
import sys
project_dir = os.path.dirname(os.getcwd())
sys.path.append(project_dir)

In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
seed = torch.Generator().manual_seed(42)
print(device)

cuda


# Data Ingestion

In [3]:
from data.cifar10 import get_cifar10_pipeline

train_loader, val_loader, test_loader = get_cifar10_pipeline(batch_size=32)
for i, data in enumerate(train_loader, 0):
    inputs, labels = data
    if i == 0:
        print("Shape of training batch inputs:", inputs.shape)
        print("Shape of training batch labels:", labels.shape)
        break
for i, data in enumerate(val_loader, 0):
    inputs, labels = data
    if i == 0:
        print("Shape of validation batch inputs:", inputs.shape)
        print("Shape of validation batch labels:", labels.shape)
        break
for i, data in enumerate(test_loader, 0):
    inputs, labels = data
    if i == 0:
        print("Shape of testing batch inputs:", inputs.shape)
        print("Shape of testing batch labels:", labels.shape)
        break
print("CIFAR-10 dataset loaded into PyTorch DataLoaders.")

Files already downloaded and verified
Files already downloaded and verified
Shape of training batch inputs: torch.Size([32, 3, 32, 32])
Shape of training batch labels: torch.Size([32])
Shape of validation batch inputs: torch.Size([32, 3, 32, 32])
Shape of validation batch labels: torch.Size([32])
Shape of testing batch inputs: torch.Size([32, 3, 32, 32])
Shape of testing batch labels: torch.Size([32])
CIFAR-10 dataset loaded into PyTorch DataLoaders.


# Model

In [4]:
import torch
import torch.optim as optim
import models.resnet as resnet
from utils.summary import get_model_stats

teacher = resnet.resnet34(num_classes=10).to(device)
student = resnet.resnet50(num_classes=10).to(device)

# note batch size and input shape influence FLOPS and params
sample = torch.randn(16, 3, 32, 32).to(device)
with torch.no_grad():
    print("Teacher model stats:")
    pred = teacher(sample)
    for name, param in get_model_stats(teacher, sample.shape).items():
        print(f"{name}: {param}")
    print("Student model stats:")
    pred = teacher(sample)
    for name, param in get_model_stats(student, sample.shape).items():
        print(f"{name}: {param}")

Teacher model stats:


Unsupported operator aten::max_pool2d encountered 1 time(s)
Unsupported operator aten::add_ encountered 16 time(s)
Unsupported operator aten::max_pool2d encountered 1 time(s)
Unsupported operator aten::add_ encountered 16 time(s)


flops: 1198694400
params: 21289802
Student model stats:
flops: 1342193664
params: 23528522
